## Consensus Scoring Function

In [1]:
def Score(s, DNA, k):
    """ 
        compute the consensus SCORE of a given k-mer 
        alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore
            DNA = list of nucleotide strings
            k = Target Motif length
    """
    score = 0
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        score += max(cnt.values())
    return score

## And here's the Score we're looking for...

In [4]:
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']

In [3]:
print(Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10))

89


In [4]:
%timeit Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10)

39.2 µs ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
print(len("CTTTGTCATCCTCCTTACTTATAGCAAGCAGTCGTCACCGGCTTGCTGAACCAACAGCTATCTGTACGGATTTGAGATTGCATAGGTGACTAATCTAACAGTGCTCATGACTGTCTTTCCGATTTTGATTGGCCTTGTGGTGTGCGTTACGTTGGTGACATATCCGTCTGTCGGATGTACTCTTTCGCAGATGTACACCGCTGATCCTCGTTGCAGCGAGTTATGTTACGTGGGGAAGTGAGCAAGTACT"))

250


## Recursive Exploration of a Search Tree

In [13]:
bestAlignment = []
prunedPaths = 0

def exploreMotifs(DNA,k,path,bestScore):
    """ Search for a k-length motif in the list of DNA sequences by exploring
        all paths in a search tree. Each call extends path by one. Once the
        path reaches the number of DNA strings a score is computed. """
    global bestAlignment, prunedPaths
    depth = len(path)
    M = len(DNA)
    if (depth == M):            # here we have an index in all M sequences
        s = Score(path,DNA,k)
        if (s > bestScore):
            bestAlignment = [p for p in path]
            return s
        else:
            return bestScore
    else:
        # Let's consider if an optimistic best score can beat the best score so far
        if (depth > 1):
            OptimisticScore = k*(M-depth) + Score(path,DNA,k)
        else:
            OptimisticScore = k*M
        if (OptimisticScore < bestScore):
            prunedPaths = prunedPaths + 1
            return bestScore
        else:
            for s in range(len(DNA[depth])-k+1):
                newPath = tuple([i for i in path] + [s])
                bestScore = exploreMotifs(DNA,k,newPath,bestScore)
            return bestScore

In [35]:
def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif, minHammingDist)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer  

In [36]:
dna = loadFasta("../PS2/motifs.fa")[1]
ContainedMotifSearch(dna, 5)

G
T
C
A
T
C
C
T
C
C
T
T
A
C
T
T
A
T
A
G
C
A
A
G
C
A
G
T
C
G
T
C
A
C
C
G
G
C
T
T
G
C
T
G
A
A
C
C
A
A
C
A
G
C
T
A
T
C
T
G
T
A
C
G
G
A
T
T
T
G
A
G
A
T
T
G
C
A
T
A
G
G
T
G
A
C
T
A
A
T
C
T
A
A
C
A
G
T
G
C
T
C
A
T
G
A
C
T
G
T
C
T
T
T
C
C
G
A
T
T
T
T
G
A
T
T
G
G
C
C
T
T
G
T
G
G
T
G
T
G
C
G
T
T
A
C
G
T
T
G
G
T
G
A
C
A
T
A
T
C
C
G
T
C
T
G
T
C
G
G
A
T
G
T
A
C
T
C
T
T
T
C
G
C
A
G
A
T
G
T
A
C
A
C
C
G
C
T
G
A
T
C
C
T
C
G
T
T
G
C
A
G
C
G
A
G
T
T
A
T
G
T
T
A
C
G
T
G
G
G
G
A
A
G
T
G
A
G
C
A
A
G
T
A
C
T


IndexError: string index out of range

## Let's try it

In [46]:
def BranchAndBoundMotifSearch(DNA, k):
    """ Finds a k-length motif within a list of DNA sequences"""
    global bestAlignment, prunedPaths
    bestAlignment = []
    prunedPaths = 0
    bestScore = 0
    bestScore = exploreMotifs(DNA,k,[],bestScore)
    print(bestAlignment, bestScore, prunedPaths)

%time BranchAndBoundMotifSearch(seqApprox[0:6], 10)

KeyboardInterrupt: 

## Scanning-and-Scoring a Motif

In [7]:
def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

In [9]:
print(ScanAndScoreMotif(seqApprox, "tagatccgaa"))
%timeit ScanAndScoreMotif(seqApprox, "tagatccgaa")

([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11)
1.45 ms ± 24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Let's do it

In [ ]:
import itertools

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('acgt', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

%time MedianStringMotifSearch(seqApprox,10)

# Let's consider only Motifs seen in the DNA

In [39]:
def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

%time ContainedMotifSearch(seqApprox,10)

709 Motifs in our set
CPU times: user 771 ms, sys: 0 ns, total: 771 ms
Wall time: 769 ms


([17, 31, 18, 33, 21, 0, 46, 70, 16, 65], 17, 'tagatccaaa')

## Contained Consensus Motif Search

In [42]:
def Consensus(s, DNA, k):
    """ compute the consensus k-Motif of an alignment given offsets into each DNA string.
            s = list of starting indices, 1-based, 0 means ignore, DNA = list of nucleotide strings,
            k = Target Motif length """
    consensus = ''
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            # loop over DNA strands
            base = DNA[j][sval+i] 
            cnt[base] += 1
        consensus += max(cnt.items(), key=lambda tup: tup[1])[0]
    return consensus

def ContainedConsensusMotifSearch(DNA,k):
    bestAlignment, minHammingDist, kmer = ContainedMotifSearch(DNA,k)
    motif = Consensus(bestAlignment,DNA,k)
    newAlignment, HammingDist = ScanAndScoreMotif(DNA, motif)
    return newAlignment, HammingDist, motif

%time ContainedConsensusMotifSearch(seqApprox,10)

709 Motifs in our set
CPU times: user 770 ms, sys: 0 ns, total: 770 ms
Wall time: 767 ms


([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')

## Randomized Motif Search

In [56]:
import random

def RandomizedMotifSearch(DNA,k):
    """ Searches for a k-length motif that appears 
    in all given DNA sequences. It begins with a
    random set of candidate consensus motifs 
    derived from the data. It refines the motif
    until a true consensus emerges."""
    
    # Seed with motifs from random alignments
    motifSet = set()
    for i in range(500):
        randomAlignment = [random.randint(0,len(DNA[j])-k) for j in range(len(DNA))]
        motif = Consensus(randomAlignment, DNA, k)
        motifSet.add(motif)

    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    testSet = motifSet.copy()
    while (len(testSet) > 0):
        print(len(motifSet),end=', ')
        nextSet = set()
        for motif in testSet:
            align, dist = ScanAndScoreMotif(DNA, motif)
            # add new motifs based on these alignments
            newMotif = Consensus(align, DNA, k)
            if (newMotif not in motifSet):
                nextSet.add(newMotif)
            if (dist < minHammingDist):
                bestAlignment = [s for s in align]
                minHammingDist = dist
                kmer = motif
        testSet = nextSet.copy()
        motifSet = motifSet | nextSet
    return bestAlignment, minHammingDist, kmer

## Let's try it

In [57]:
%time RandomizedMotifSearch(seqApprox,10)

500, 771, 866, 883, 889, 890, CPU times: user 1.03 s, sys: 4.01 ms, total: 1.04 s
Wall time: 1.03 s


([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')

In [58]:
for i in range(10):
    print(RandomizedMotifSearch(seqApprox,10))

499, 774, 861, 876, 878, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 768, 843, 863, 869, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 743, 823, 843, 845, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 756, 832, 844, 845, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 745, 826, 844, 850, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 776, 852, 870, 873, 874, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 762, 857, 878, 880, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 753, 822, 839, 844, 845, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 764, 845, 865, 867, 868, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')
500, 749, 825, 839, 841, ([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], 11, 'tagatccgaa')


In [12]:
# THIS CELL WILL BE GRADED
# This cell should include *ALL* functions called by BestMedianMotifSearch(),
# such as ContainedMedianSearch() and ScanAndScoreMotif(). Any test or 
# validation code should be placed in another cell 

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment1 = []
    minHammingDist1 = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif, minHammingDist1)
        if (dist < minHammingDist1):
            bestAlignment1 = [s for s in align]
            minHammingDist1 = dist
            kmer = motif
    return bestAlignment1, minHammingDist1, kmer

def ScanAndScoreMotif(DNA, motif, dist):
    totalDist = 0
    bestAlignment2 = []
    k = len(motif)
    for seq in DNA:
        minHammingDist2 = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist2):
                bestS = s
                minHammingDist2 = HammingDist
        bestAlignment2.append(bestS)
        totalDist += minHammingDist2
    return bestAlignment2, totalDist

def getMotif(DNA, k, motif, minHammingDist):
    global bestAlignment, globalKmer
    depth = len(motif)
    if (depth == k):
        alignment, s = ScanAndScoreMotif(DNA, motif, minHammingDist)
        if (s < minHammingDist):
            bestAlignment = [a for a in alignment]
            globalKmer = motif
            return s
        else:
            return minHammingDist
    else:
        bestAlignment, OptimisticScore = ScanAndScoreMotif(DNA, motif, minHammingDist)
        if (OptimisticScore > minHammingDist):
            return minHammingDist
        else:
            for base in 'ACGT':
                motif = motif+base
                minHammingDist = getMotif(DNA, k, motif, minHammingDist) 
            return minHammingDist

def BestMedianMotifSearch(DNA,k):
    global bestAlignment, globalKmer, minHammingDist
    bestAlignment, minHammingDist, globalKmer = ContainedMotifSearch(DNA, k)
    getMotif(DNA, k, '', minHammingDist)
    return bestAlignment, minHammingDist, globalKmer

In [13]:
# This cell is provided for testing your answer to problem 5. It will not be graded.
dna = loadFasta("../PS2/motifs.fa")[1]
dna = [d[1:] for d in dna]
%time BestMedianMotifSearch(dna,5)

1003 Motifs in our set
Wall time: 6.99 s


([148, 111, 57, 113, 93, 11, 7, 86, 160, 170, 22, 152, 42, 113, 91, 64],
 6,
 'CCGCT')